# Final pre-processing file

! pip install deep-translator
! pip install langid

In [1]:
! pip install deep-translator
! pip install langid

  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941187 sha256=da3a44263db4da971b296e01c45836f0212942460755c20c1c42ac98adc71057
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\93\95\a9\c292c9dd8cadb8f2359f1670ff198a40d47167b0be3236e1c8
Successfully built langid


In [2]:
import numpy as np
import pandas as pd
import re
from functools import partial
from collections import Counter
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import langid
#import googletrans
#from googletrans import Translator
from deep_translator import GoogleTranslator 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv(r'C:/Users/DELL/Downloads/TCS senti.csv',encoding= 'unicode_escape')
print(df)

             Date      Time  Unnamed: 2             User  \
0      08-01-2016  23:02:04         NaN   PolskieRadio24   
1      08-01-2016  23:02:04         NaN       el_dudkers   
2      15-01-2016  22:18:03         NaN         atkowska   
3      15-01-2016  22:18:03         NaN     BikeTeesside   
4      19-01-2016  06:03:35         NaN       TCStrength   
...           ...       ...         ...              ...   
73798  30-12-2019  22:28:31         NaN         xintae08   
73799  30-12-2019  22:28:31         NaN         xintae08   
73800  30-12-2019  22:46:29         NaN      chicapotter   
73801  30-12-2019  23:23:41         NaN        balgeun_i   
73802  30-12-2019  23:24:25         NaN  ThakurEntrprses   

                                                   Tweet  \
0      Turniej Czterech Skoczni: Kamil #Stoch drugi w...   
1      Super ten konkurs w Oberstdorfie. Znakomicie P...   
2      https://t.co/06EaOW4Zdg - udanie dziÅ w #Ober...   
3      Old standard but always the best

In [5]:
df1 = df.copy()

In [6]:
df1['Tweet'].isnull().sum()

0

In [7]:
data = df1.drop_duplicates(keep = 'first')

In [8]:
data

,Date,Time,Unnamed: 2,User,Tweet,Tweet_created,Url,Location,Friends_count,Favourites_Count,Media,Reply_count,Retweet_count,Retweeted_tweet,Followers_count,Hashtags
0,08-01-2016,23:02:04,NaN,PolskieRadio24,Turniej Czterech Skoczni: Kamil #Stoch drugi w...,2013-12-10 11:31:33+00:00,https://twitter.com/PolskieRadio24/status/8149...,NaN,1466,9836,NaN,1,5,NaN,50301,"['Stoch', 'Oberstdorf', 'Sport', 'PR24', 'TCS'..."
1,08-01-2016,23:02:04,NaN,el_dudkers,Super ten konkurs w Oberstdorfie. Znakomicie P...,2014-05-15 12:31:51+00:00,https://twitter.com/el_dudkers/status/81493475...,NaN,436,3172,NaN,0,0,NaN,121,"['Skoki', 'TCS']"
2,15-01-2016,22:18:03,NaN,atkowska,https://t.co/06EaOW4Zdg - udanie dziÅ w #Ober...,2014-05-27 17:37:57+00:00,https://twitter.com/atkowska/status/8149248702...,NaN,241,1268,NaN,0,0,NaN,154,"['Oberstdorf', 'tcs', 'skijumping']"
3,15-01-2016,22:18:03,NaN,BikeTeesside,Old standard but always the best quality spare...,2014-04-10 14:47:40+00:00,https://twitter.com/BikeTeesside/status/814915...,Teesside,181,911,[Photo(previewUrl='https://pbs.twimg.com/media...,0,0,NaN,233,"['TCS', 'bikeservice', 'teesside', 'mtb']"
4,19-01-2016,06:03:35,NaN,TCStrength,It's incredible how satisfying throwing heavy ...,2011-02-22 23:23:10+00:00,https://twitter.com/TCStrength/status/81491031...,"Austin, TX",273,5,NaN,0,0,NaN,1690,['tcs']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73798,30-12-2019,22:28:31,NaN,xintae08,year-ender? ð¤§\n#Tata #Taetae https://t.co/Z...,2017-03-21 03:34:45+00:00,https://twitter.com/xintae08/status/1211776105...,"Auckland, New Zealand",1289,31729,[Photo(previewUrl='https://pbs.twimg.com/media...,1,0,NaN,1165,"['Tata', 'Taetae']"
73799,30-12-2019,22:28:31,NaN,xintae08,year-ender? ð¤§\n#Tata #Taetae https://t.co/Z...,2017-03-21 03:34:45+00:00,https://twitter.com/xintae08/status/1211776105...,"Auckland, New Zealand",1289,31729,[Photo(previewUrl='https://pbs.twimg.com/media...,1,0,NaN,1164,"['Tata', 'Taetae']"
73800,30-12-2019,22:46:29,NaN,chicapotter,"Hoy cumple aÃ±os bebÃ© Tae, Ã©l me robÃ³ el co...",2010-03-30 20:28:36+00:00,https://twitter.com/chicapotter/status/1211780...,Puebla,362,2378,NaN,0,0,NaN,80,"['chicapotter', 'sonyoueli', 'army', 'btsarmy'..."
73801,30-12-2019,23:23:41,NaN,balgeun_i,ë¤ ì¤ëì 2019 ë§ì§ë§ ë ì´êµ¬ì \nì...,2011-05-26 00:36:28+00:00,https://twitter.com/balgeun_i/status/121178998...,ëíë¯¼êµ­,548,26089,[Photo(previewUrl='https://pbs.twimg.com/media...,11,23,NaN,973,"['RJ', 'TATA', 'íë³µíì¸ì']"


In [9]:
data

,Date,Time,Unnamed: 2,User,Tweet,Tweet_created,Url,Location,Friends_count,Favourites_Count,Media,Reply_count,Retweet_count,Retweeted_tweet,Followers_count,Hashtags
0,08-01-2016,23:02:04,NaN,PolskieRadio24,Turniej Czterech Skoczni: Kamil #Stoch drugi w...,2013-12-10 11:31:33+00:00,https://twitter.com/PolskieRadio24/status/8149...,NaN,1466,9836,NaN,1,5,NaN,50301,"['Stoch', 'Oberstdorf', 'Sport', 'PR24', 'TCS'..."
1,08-01-2016,23:02:04,NaN,el_dudkers,Super ten konkurs w Oberstdorfie. Znakomicie P...,2014-05-15 12:31:51+00:00,https://twitter.com/el_dudkers/status/81493475...,NaN,436,3172,NaN,0,0,NaN,121,"['Skoki', 'TCS']"
2,15-01-2016,22:18:03,NaN,atkowska,https://t.co/06EaOW4Zdg - udanie dziÅ w #Ober...,2014-05-27 17:37:57+00:00,https://twitter.com/atkowska/status/8149248702...,NaN,241,1268,NaN,0,0,NaN,154,"['Oberstdorf', 'tcs', 'skijumping']"
3,15-01-2016,22:18:03,NaN,BikeTeesside,Old standard but always the best quality spare...,2014-04-10 14:47:40+00:00,https://twitter.com/BikeTeesside/status/814915...,Teesside,181,911,[Photo(previewUrl='https://pbs.twimg.com/media...,0,0,NaN,233,"['TCS', 'bikeservice', 'teesside', 'mtb']"
4,19-01-2016,06:03:35,NaN,TCStrength,It's incredible how satisfying throwing heavy ...,2011-02-22 23:23:10+00:00,https://twitter.com/TCStrength/status/81491031...,"Austin, TX",273,5,NaN,0,0,NaN,1690,['tcs']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73798,30-12-2019,22:28:31,NaN,xintae08,year-ender? ð¤§\n#Tata #Taetae https://t.co/Z...,2017-03-21 03:34:45+00:00,https://twitter.com/xintae08/status/1211776105...,"Auckland, New Zealand",1289,31729,[Photo(previewUrl='https://pbs.twimg.com/media...,1,0,NaN,1165,"['Tata', 'Taetae']"
73799,30-12-2019,22:28:31,NaN,xintae08,year-ender? ð¤§\n#Tata #Taetae https://t.co/Z...,2017-03-21 03:34:45+00:00,https://twitter.com/xintae08/status/1211776105...,"Auckland, New Zealand",1289,31729,[Photo(previewUrl='https://pbs.twimg.com/media...,1,0,NaN,1164,"['Tata', 'Taetae']"
73800,30-12-2019,22:46:29,NaN,chicapotter,"Hoy cumple aÃ±os bebÃ© Tae, Ã©l me robÃ³ el co...",2010-03-30 20:28:36+00:00,https://twitter.com/chicapotter/status/1211780...,Puebla,362,2378,NaN,0,0,NaN,80,"['chicapotter', 'sonyoueli', 'army', 'btsarmy'..."
73801,30-12-2019,23:23:41,NaN,balgeun_i,ë¤ ì¤ëì 2019 ë§ì§ë§ ë ì´êµ¬ì \nì...,2011-05-26 00:36:28+00:00,https://twitter.com/balgeun_i/status/121178998...,ëíë¯¼êµ­,548,26089,[Photo(previewUrl='https://pbs.twimg.com/media...,11,23,NaN,973,"['RJ', 'TATA', 'íë³µíì¸ì']"


##### Text to remove unicode

In [10]:
def removeUnicode(text):
    """ Removes unicode strings like "\u002c" and "x96" """
    text = re.sub(r'(\\u[0-9A-Fa-f]+)',' ', text)       
    text = re.sub(r'[^\x00-\x7f]',' ',text)
    return text

In [11]:
data['Tweet'] = data['Tweet'].apply(lambda x : removeUnicode(x))

In [12]:
data

,Date,Time,Unnamed: 2,User,Tweet,Tweet_created,Url,Location,Friends_count,Favourites_Count,Media,Reply_count,Retweet_count,Retweeted_tweet,Followers_count,Hashtags
0,08-01-2016,23:02:04,NaN,PolskieRadio24,Turniej Czterech Skoczni: Kamil #Stoch drugi w...,2013-12-10 11:31:33+00:00,https://twitter.com/PolskieRadio24/status/8149...,NaN,1466,9836,NaN,1,5,NaN,50301,"['Stoch', 'Oberstdorf', 'Sport', 'PR24', 'TCS'..."
1,08-01-2016,23:02:04,NaN,el_dudkers,Super ten konkurs w Oberstdorfie. Znakomicie P...,2014-05-15 12:31:51+00:00,https://twitter.com/el_dudkers/status/81493475...,NaN,436,3172,NaN,0,0,NaN,121,"['Skoki', 'TCS']"
2,15-01-2016,22:18:03,NaN,atkowska,https://t.co/06EaOW4Zdg - udanie dzi w #Ober...,2014-05-27 17:37:57+00:00,https://twitter.com/atkowska/status/8149248702...,NaN,241,1268,NaN,0,0,NaN,154,"['Oberstdorf', 'tcs', 'skijumping']"
3,15-01-2016,22:18:03,NaN,BikeTeesside,Old standard but always the best quality spare...,2014-04-10 14:47:40+00:00,https://twitter.com/BikeTeesside/status/814915...,Teesside,181,911,[Photo(previewUrl='https://pbs.twimg.com/media...,0,0,NaN,233,"['TCS', 'bikeservice', 'teesside', 'mtb']"
4,19-01-2016,06:03:35,NaN,TCStrength,It's incredible how satisfying throwing heavy ...,2011-02-22 23:23:10+00:00,https://twitter.com/TCStrength/status/81491031...,"Austin, TX",273,5,NaN,0,0,NaN,1690,['tcs']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73798,30-12-2019,22:28:31,NaN,xintae08,year-ender? \n#Tata #Taetae https://t.co/Z...,2017-03-21 03:34:45+00:00,https://twitter.com/xintae08/status/1211776105...,"Auckland, New Zealand",1289,31729,[Photo(previewUrl='https://pbs.twimg.com/media...,1,0,NaN,1165,"['Tata', 'Taetae']"
73799,30-12-2019,22:28:31,NaN,xintae08,year-ender? \n#Tata #Taetae https://t.co/Z...,2017-03-21 03:34:45+00:00,https://twitter.com/xintae08/status/1211776105...,"Auckland, New Zealand",1289,31729,[Photo(previewUrl='https://pbs.twimg.com/media...,1,0,NaN,1164,"['Tata', 'Taetae']"
73800,30-12-2019,22:46:29,NaN,chicapotter,"Hoy cumple a os beb Tae, l me rob el co...",2010-03-30 20:28:36+00:00,https://twitter.com/chicapotter/status/1211780...,Puebla,362,2378,NaN,0,0,NaN,80,"['chicapotter', 'sonyoueli', 'army', 'btsarmy'..."
73801,30-12-2019,23:23:41,NaN,balgeun_i,2019 \n ...,2011-05-26 00:36:28+00:00,https://twitter.com/balgeun_i/status/121178998...,ëíë¯¼êµ­,548,26089,[Photo(previewUrl='https://pbs.twimg.com/media...,11,23,NaN,973,"['RJ', 'TATA', 'íë³µíì¸ì']"


#### Replace URL

In [13]:
def replaceURL(text):
    """ Replaces url address with "url" """
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)
    text = re.sub(r'#([^\s]+)', r' ', text)
    return text

In [14]:
data['Tweet'] = data['Tweet'].apply(lambda x : replaceURL(x))

In [15]:
data

,Date,Time,Unnamed: 2,User,Tweet,Tweet_created,Url,Location,Friends_count,Favourites_Count,Media,Reply_count,Retweet_count,Retweeted_tweet,Followers_count,Hashtags
0,08-01-2016,23:02:04,NaN,PolskieRadio24,Turniej Czterech Skoczni: Kamil drugi w ...,2013-12-10 11:31:33+00:00,https://twitter.com/PolskieRadio24/status/8149...,NaN,1466,9836,NaN,1,5,NaN,50301,"['Stoch', 'Oberstdorf', 'Sport', 'PR24', 'TCS'..."
1,08-01-2016,23:02:04,NaN,el_dudkers,Super ten konkurs w Oberstdorfie. Znakomicie P...,2014-05-15 12:31:51+00:00,https://twitter.com/el_dudkers/status/81493475...,NaN,436,3172,NaN,0,0,NaN,121,"['Skoki', 'TCS']"
2,15-01-2016,22:18:03,NaN,atkowska,- udanie dzi w a tak by o prawie ju 1...,2014-05-27 17:37:57+00:00,https://twitter.com/atkowska/status/8149248702...,NaN,241,1268,NaN,0,0,NaN,154,"['Oberstdorf', 'tcs', 'skijumping']"
3,15-01-2016,22:18:03,NaN,BikeTeesside,Old standard but always the best quality spare...,2014-04-10 14:47:40+00:00,https://twitter.com/BikeTeesside/status/814915...,Teesside,181,911,[Photo(previewUrl='https://pbs.twimg.com/media...,0,0,NaN,233,"['TCS', 'bikeservice', 'teesside', 'mtb']"
4,19-01-2016,06:03:35,NaN,TCStrength,It's incredible how satisfying throwing heavy ...,2011-02-22 23:23:10+00:00,https://twitter.com/TCStrength/status/81491031...,"Austin, TX",273,5,NaN,0,0,NaN,1690,['tcs']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73798,30-12-2019,22:28:31,NaN,xintae08,year-ender? \n,2017-03-21 03:34:45+00:00,https://twitter.com/xintae08/status/1211776105...,"Auckland, New Zealand",1289,31729,[Photo(previewUrl='https://pbs.twimg.com/media...,1,0,NaN,1165,"['Tata', 'Taetae']"
73799,30-12-2019,22:28:31,NaN,xintae08,year-ender? \n,2017-03-21 03:34:45+00:00,https://twitter.com/xintae08/status/1211776105...,"Auckland, New Zealand",1289,31729,[Photo(previewUrl='https://pbs.twimg.com/media...,1,0,NaN,1164,"['Tata', 'Taetae']"
73800,30-12-2019,22:46:29,NaN,chicapotter,"Hoy cumple a os beb Tae, l me rob el co...",2010-03-30 20:28:36+00:00,https://twitter.com/chicapotter/status/1211780...,Puebla,362,2378,NaN,0,0,NaN,80,"['chicapotter', 'sonyoueli', 'army', 'btsarmy'..."
73801,30-12-2019,23:23:41,NaN,balgeun_i,2019 \n ...,2011-05-26 00:36:28+00:00,https://twitter.com/balgeun_i/status/121178998...,ëíë¯¼êµ­,548,26089,[Photo(previewUrl='https://pbs.twimg.com/media...,11,23,NaN,973,"['RJ', 'TATA', 'íë³µíì¸ì']"


#### Replace @ user

In [16]:
def replaceAtUser(text):
    """ Replaces "@user" with "atUser" """
    text = re.sub('@[^\s]+',' ',text)
    return text

In [17]:
data['Tweet'] = data['Tweet'].apply(lambda x : replaceAtUser(x))

In [18]:
data

,Date,Time,Unnamed: 2,User,Tweet,Tweet_created,Url,Location,Friends_count,Favourites_Count,Media,Reply_count,Retweet_count,Retweeted_tweet,Followers_count,Hashtags
0,08-01-2016,23:02:04,NaN,PolskieRadio24,Turniej Czterech Skoczni: Kamil drugi w ...,2013-12-10 11:31:33+00:00,https://twitter.com/PolskieRadio24/status/8149...,NaN,1466,9836,NaN,1,5,NaN,50301,"['Stoch', 'Oberstdorf', 'Sport', 'PR24', 'TCS'..."
1,08-01-2016,23:02:04,NaN,el_dudkers,Super ten konkurs w Oberstdorfie. Znakomicie P...,2014-05-15 12:31:51+00:00,https://twitter.com/el_dudkers/status/81493475...,NaN,436,3172,NaN,0,0,NaN,121,"['Skoki', 'TCS']"
2,15-01-2016,22:18:03,NaN,atkowska,- udanie dzi w a tak by o prawie ju 1...,2014-05-27 17:37:57+00:00,https://twitter.com/atkowska/status/8149248702...,NaN,241,1268,NaN,0,0,NaN,154,"['Oberstdorf', 'tcs', 'skijumping']"
3,15-01-2016,22:18:03,NaN,BikeTeesside,Old standard but always the best quality spare...,2014-04-10 14:47:40+00:00,https://twitter.com/BikeTeesside/status/814915...,Teesside,181,911,[Photo(previewUrl='https://pbs.twimg.com/media...,0,0,NaN,233,"['TCS', 'bikeservice', 'teesside', 'mtb']"
4,19-01-2016,06:03:35,NaN,TCStrength,It's incredible how satisfying throwing heavy ...,2011-02-22 23:23:10+00:00,https://twitter.com/TCStrength/status/81491031...,"Austin, TX",273,5,NaN,0,0,NaN,1690,['tcs']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73798,30-12-2019,22:28:31,NaN,xintae08,year-ender? \n,2017-03-21 03:34:45+00:00,https://twitter.com/xintae08/status/1211776105...,"Auckland, New Zealand",1289,31729,[Photo(previewUrl='https://pbs.twimg.com/media...,1,0,NaN,1165,"['Tata', 'Taetae']"
73799,30-12-2019,22:28:31,NaN,xintae08,year-ender? \n,2017-03-21 03:34:45+00:00,https://twitter.com/xintae08/status/1211776105...,"Auckland, New Zealand",1289,31729,[Photo(previewUrl='https://pbs.twimg.com/media...,1,0,NaN,1164,"['Tata', 'Taetae']"
73800,30-12-2019,22:46:29,NaN,chicapotter,"Hoy cumple a os beb Tae, l me rob el co...",2010-03-30 20:28:36+00:00,https://twitter.com/chicapotter/status/1211780...,Puebla,362,2378,NaN,0,0,NaN,80,"['chicapotter', 'sonyoueli', 'army', 'btsarmy'..."
73801,30-12-2019,23:23:41,NaN,balgeun_i,2019 \n ...,2011-05-26 00:36:28+00:00,https://twitter.com/balgeun_i/status/121178998...,ëíë¯¼êµ­,548,26089,[Photo(previewUrl='https://pbs.twimg.com/media...,11,23,NaN,973,"['RJ', 'TATA', 'íë³µíì¸ì']"


#### Remove Numbers

In [19]:
def removeNumbers(text):
    """ Removes integers """
    text = ''.join([i for i in text if not i.isdigit()])         
    return text

In [20]:
data['Tweet'] = data['Tweet'].apply(lambda x : removeNumbers(x))

In [21]:
data

,Date,Time,Unnamed: 2,User,Tweet,Tweet_created,Url,Location,Friends_count,Favourites_Count,Media,Reply_count,Retweet_count,Retweeted_tweet,Followers_count,Hashtags
0,08-01-2016,23:02:04,NaN,PolskieRadio24,Turniej Czterech Skoczni: Kamil drugi w ...,2013-12-10 11:31:33+00:00,https://twitter.com/PolskieRadio24/status/8149...,NaN,1466,9836,NaN,1,5,NaN,50301,"['Stoch', 'Oberstdorf', 'Sport', 'PR24', 'TCS'..."
1,08-01-2016,23:02:04,NaN,el_dudkers,Super ten konkurs w Oberstdorfie. Znakomicie P...,2014-05-15 12:31:51+00:00,https://twitter.com/el_dudkers/status/81493475...,NaN,436,3172,NaN,0,0,NaN,121,"['Skoki', 'TCS']"
2,15-01-2016,22:18:03,NaN,atkowska,- udanie dzi w a tak by o prawie ju ...,2014-05-27 17:37:57+00:00,https://twitter.com/atkowska/status/8149248702...,NaN,241,1268,NaN,0,0,NaN,154,"['Oberstdorf', 'tcs', 'skijumping']"
3,15-01-2016,22:18:03,NaN,BikeTeesside,Old standard but always the best quality spare...,2014-04-10 14:47:40+00:00,https://twitter.com/BikeTeesside/status/814915...,Teesside,181,911,[Photo(previewUrl='https://pbs.twimg.com/media...,0,0,NaN,233,"['TCS', 'bikeservice', 'teesside', 'mtb']"
4,19-01-2016,06:03:35,NaN,TCStrength,It's incredible how satisfying throwing heavy ...,2011-02-22 23:23:10+00:00,https://twitter.com/TCStrength/status/81491031...,"Austin, TX",273,5,NaN,0,0,NaN,1690,['tcs']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73798,30-12-2019,22:28:31,NaN,xintae08,year-ender? \n,2017-03-21 03:34:45+00:00,https://twitter.com/xintae08/status/1211776105...,"Auckland, New Zealand",1289,31729,[Photo(previewUrl='https://pbs.twimg.com/media...,1,0,NaN,1165,"['Tata', 'Taetae']"
73799,30-12-2019,22:28:31,NaN,xintae08,year-ender? \n,2017-03-21 03:34:45+00:00,https://twitter.com/xintae08/status/1211776105...,"Auckland, New Zealand",1289,31729,[Photo(previewUrl='https://pbs.twimg.com/media...,1,0,NaN,1164,"['Tata', 'Taetae']"
73800,30-12-2019,22:46:29,NaN,chicapotter,"Hoy cumple a os beb Tae, l me rob el co...",2010-03-30 20:28:36+00:00,https://twitter.com/chicapotter/status/1211780...,Puebla,362,2378,NaN,0,0,NaN,80,"['chicapotter', 'sonyoueli', 'army', 'btsarmy'..."
73801,30-12-2019,23:23:41,NaN,balgeun_i,\n ...,2011-05-26 00:36:28+00:00,https://twitter.com/balgeun_i/status/121178998...,ëíë¯¼êµ­,548,26089,[Photo(previewUrl='https://pbs.twimg.com/media...,11,23,NaN,973,"['RJ', 'TATA', 'íë³µíì¸ì']"


#### Replace Multi exclamation

In [22]:
def replaceMultiExclamationMark(text):
    """ Replaces repetitions of exlamation marks """
    text = re.sub(r"(\!)\1+", ' ', text)
    return text

In [23]:
data['Tweet'] = data['Tweet'].apply(lambda x : replaceMultiExclamationMark(x))

In [24]:
data

,Date,Time,Unnamed: 2,User,Tweet,Tweet_created,Url,Location,Friends_count,Favourites_Count,Media,Reply_count,Retweet_count,Retweeted_tweet,Followers_count,Hashtags
0,08-01-2016,23:02:04,NaN,PolskieRadio24,Turniej Czterech Skoczni: Kamil drugi w ...,2013-12-10 11:31:33+00:00,https://twitter.com/PolskieRadio24/status/8149...,NaN,1466,9836,NaN,1,5,NaN,50301,"['Stoch', 'Oberstdorf', 'Sport', 'PR24', 'TCS'..."
1,08-01-2016,23:02:04,NaN,el_dudkers,Super ten konkurs w Oberstdorfie. Znakomicie P...,2014-05-15 12:31:51+00:00,https://twitter.com/el_dudkers/status/81493475...,NaN,436,3172,NaN,0,0,NaN,121,"['Skoki', 'TCS']"
2,15-01-2016,22:18:03,NaN,atkowska,- udanie dzi w a tak by o prawie ju ...,2014-05-27 17:37:57+00:00,https://twitter.com/atkowska/status/8149248702...,NaN,241,1268,NaN,0,0,NaN,154,"['Oberstdorf', 'tcs', 'skijumping']"
3,15-01-2016,22:18:03,NaN,BikeTeesside,Old standard but always the best quality spare...,2014-04-10 14:47:40+00:00,https://twitter.com/BikeTeesside/status/814915...,Teesside,181,911,[Photo(previewUrl='https://pbs.twimg.com/media...,0,0,NaN,233,"['TCS', 'bikeservice', 'teesside', 'mtb']"
4,19-01-2016,06:03:35,NaN,TCStrength,It's incredible how satisfying throwing heavy ...,2011-02-22 23:23:10+00:00,https://twitter.com/TCStrength/status/81491031...,"Austin, TX",273,5,NaN,0,0,NaN,1690,['tcs']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73798,30-12-2019,22:28:31,NaN,xintae08,year-ender? \n,2017-03-21 03:34:45+00:00,https://twitter.com/xintae08/status/1211776105...,"Auckland, New Zealand",1289,31729,[Photo(previewUrl='https://pbs.twimg.com/media...,1,0,NaN,1165,"['Tata', 'Taetae']"
73799,30-12-2019,22:28:31,NaN,xintae08,year-ender? \n,2017-03-21 03:34:45+00:00,https://twitter.com/xintae08/status/1211776105...,"Auckland, New Zealand",1289,31729,[Photo(previewUrl='https://pbs.twimg.com/media...,1,0,NaN,1164,"['Tata', 'Taetae']"
73800,30-12-2019,22:46:29,NaN,chicapotter,"Hoy cumple a os beb Tae, l me rob el co...",2010-03-30 20:28:36+00:00,https://twitter.com/chicapotter/status/1211780...,Puebla,362,2378,NaN,0,0,NaN,80,"['chicapotter', 'sonyoueli', 'army', 'btsarmy'..."
73801,30-12-2019,23:23:41,NaN,balgeun_i,\n ...,2011-05-26 00:36:28+00:00,https://twitter.com/balgeun_i/status/121178998...,ëíë¯¼êµ­,548,26089,[Photo(previewUrl='https://pbs.twimg.com/media...,11,23,NaN,973,"['RJ', 'TATA', 'íë³µíì¸ì']"


#### Replace Multi questionmark

In [25]:
def replaceMultiQuestionMark(text):
    """ Replaces repetitions of question marks """
    text = re.sub(r"(\?)\1+", ' ', text)
    return text

In [26]:
data['Tweet'] = data['Tweet'].apply(lambda x : replaceMultiQuestionMark(x))

In [27]:
data

,Date,Time,Unnamed: 2,User,Tweet,Tweet_created,Url,Location,Friends_count,Favourites_Count,Media,Reply_count,Retweet_count,Retweeted_tweet,Followers_count,Hashtags
0,08-01-2016,23:02:04,NaN,PolskieRadio24,Turniej Czterech Skoczni: Kamil drugi w ...,2013-12-10 11:31:33+00:00,https://twitter.com/PolskieRadio24/status/8149...,NaN,1466,9836,NaN,1,5,NaN,50301,"['Stoch', 'Oberstdorf', 'Sport', 'PR24', 'TCS'..."
1,08-01-2016,23:02:04,NaN,el_dudkers,Super ten konkurs w Oberstdorfie. Znakomicie P...,2014-05-15 12:31:51+00:00,https://twitter.com/el_dudkers/status/81493475...,NaN,436,3172,NaN,0,0,NaN,121,"['Skoki', 'TCS']"
2,15-01-2016,22:18:03,NaN,atkowska,- udanie dzi w a tak by o prawie ju ...,2014-05-27 17:37:57+00:00,https://twitter.com/atkowska/status/8149248702...,NaN,241,1268,NaN,0,0,NaN,154,"['Oberstdorf', 'tcs', 'skijumping']"
3,15-01-2016,22:18:03,NaN,BikeTeesside,Old standard but always the best quality spare...,2014-04-10 14:47:40+00:00,https://twitter.com/BikeTeesside/status/814915...,Teesside,181,911,[Photo(previewUrl='https://pbs.twimg.com/media...,0,0,NaN,233,"['TCS', 'bikeservice', 'teesside', 'mtb']"
4,19-01-2016,06:03:35,NaN,TCStrength,It's incredible how satisfying throwing heavy ...,2011-02-22 23:23:10+00:00,https://twitter.com/TCStrength/status/81491031...,"Austin, TX",273,5,NaN,0,0,NaN,1690,['tcs']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73798,30-12-2019,22:28:31,NaN,xintae08,year-ender? \n,2017-03-21 03:34:45+00:00,https://twitter.com/xintae08/status/1211776105...,"Auckland, New Zealand",1289,31729,[Photo(previewUrl='https://pbs.twimg.com/media...,1,0,NaN,1165,"['Tata', 'Taetae']"
73799,30-12-2019,22:28:31,NaN,xintae08,year-ender? \n,2017-03-21 03:34:45+00:00,https://twitter.com/xintae08/status/1211776105...,"Auckland, New Zealand",1289,31729,[Photo(previewUrl='https://pbs.twimg.com/media...,1,0,NaN,1164,"['Tata', 'Taetae']"
73800,30-12-2019,22:46:29,NaN,chicapotter,"Hoy cumple a os beb Tae, l me rob el co...",2010-03-30 20:28:36+00:00,https://twitter.com/chicapotter/status/1211780...,Puebla,362,2378,NaN,0,0,NaN,80,"['chicapotter', 'sonyoueli', 'army', 'btsarmy'..."
73801,30-12-2019,23:23:41,NaN,balgeun_i,\n ...,2011-05-26 00:36:28+00:00,https://twitter.com/balgeun_i/status/121178998...,ëíë¯¼êµ­,548,26089,[Photo(previewUrl='https://pbs.twimg.com/media...,11,23,NaN,973,"['RJ', 'TATA', 'íë³µíì¸ì']"


In [28]:
import string

In [29]:
data['Tweet'] = data['Tweet'].str.replace('\n',' ') # To remove '\n' from every review

In [30]:
data.sample(20)

,Date,Time,Unnamed: 2,User,Tweet,Tweet_created,Url,Location,Friends_count,Favourites_Count,Media,Reply_count,Retweet_count,Retweeted_tweet,Followers_count,Hashtags
43321,29-11-2017,01:29:10,NaN,MrsQuadagnoTCS,preps for night with th graders! Great wor...,2015-12-06 19:22:43+00:00,https://twitter.com/MrsQuadagnoTCS/status/9356...,NaN,163,1549,[Photo(previewUrl='https://pbs.twimg.com/media...,0,0,NaN,189,"['TCSMS', 'exhibition', 'TCS']"
67994,04-12-2019,05:00:13,NaN,WeLoveSF,...,2009-09-03 09:31:55+00:00,https://twitter.com/WeLoveSF/status/1202090206...,Thailand,73,16,[Photo(previewUrl='https://pbs.twimg.com/media...,1,10,NaN,1475893,"['SFcinema', 'BT21', 'KOYA', 'RJ', 'SHOOKY', '..."
33575,27-12-2016,06:04:29,NaN,business_supser,Cyrus Mistry seeks shareholders support to sta...,2016-02-01 19:32:51+00:00,https://twitter.com/business_supser/status/807...,"San Mateo, CA",2,0,NaN,0,0,NaN,2108,['Tata']
42906,25-11-2017,23:46:24,NaN,DisguiseDevilin,"Un autre au ,FM vient de d clarer qu'ici, n...",2015-12-22 21:52:34+00:00,https://twitter.com/DisguiseDevilin/status/934...,Terre/Earth,4932,69870,NaN,0,0,NaN,3442,"['Tata', 'YÃ©PerduLui', 'DescripteurMatchsCH']"
66563,25-11-2019,13:40:20,NaN,ForzaHana,. # ...,2017-09-01 12:22:25+00:00,https://twitter.com/ForzaHana/status/119895960...,æ¥æ¬,326,118122,[Photo(previewUrl='https://pbs.twimg.com/media...,0,1,NaN,1642,"['æ¨æè±', 'TCS', 'ã¹ã¿ã¼ãã ']"
40737,15-11-2017,16:38:37,NaN,ljilja_susnjar,"Amazing program,excellent system where everybo...",2011-04-29 10:09:45+00:00,https://twitter.com/ljilja_susnjar/status/9308...,Bosnien and Herzegovina,800,83,[Photo(previewUrl='https://pbs.twimg.com/media...,0,0,NaN,297,"['matrix2cycler', 'm2c', 'trafficcircleshare',..."
56153,10-12-2018,00:55:44,NaN,snow0517t,# # # ...,2012-09-21 13:47:56+00:00,https://twitter.com/snow0517t/status/107193143...,"Taichung City, Taiwan",26,1274,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0,0,NaN,7,"['é²å½å°å¹´å', 'éç¢©ç', 'éåä¿', ..."
71806,22-12-2019,09:59:46,NaN,Dev_Discourse,promoted to soon raise USD - mn,2017-12-05 08:31:31+00:00,https://twitter.com/Dev_Discourse/status/12086...,"Sonipat - Delhi, NCR",598,29967,NaN,0,0,NaN,208475,"['Tata', 'Nilekani', 'AvantiFinance']"
64307,12-11-2019,14:24:17,NaN,Pradeepmeerut,may bought btw - Sl wcb st target nd tgt ...,2016-02-13 06:43:00+00:00,https://twitter.com/Pradeepmeerut/status/11942...,"Meerut,India",511,1554,NaN,0,0,NaN,58,"['Tcs', 'tcs', 'share', 'nifty', 'NIFTYFUTURE'..."
71436,20-12-2019,04:22:52,NaN,keetabambam,BT ...,2015-12-19 14:13:52+00:00,https://twitter.com/keetabambam/status/1207879...,NaN,355,23142,[Photo(previewUrl='https://pbs.twimg.com/media...,0,0,NaN,7022,"['Tata', 'chimmy', 'à¸à¸¥à¸²à¸à¸à¸±à¸à¸à¸..."


In [31]:
! pip install emot

In [32]:
import emot
from emot.emo_unicode import UNICODE_EMOJI

In [33]:
def convert_emojis(text):
    for emot in UNICODE_EMOJI:
        text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
    return text

In [34]:
data['Tweet'] = data['Tweet'].apply(lambda x : convert_emojis(x))

In [35]:
data

,Date,Time,Unnamed: 2,User,Tweet,Tweet_created,Url,Location,Friends_count,Favourites_Count,Media,Reply_count,Retweet_count,Retweeted_tweet,Followers_count,Hashtags
0,08-01-2016,23:02:04,NaN,PolskieRadio24,Turniej Czterech Skoczni: Kamil drugi w ...,2013-12-10 11:31:33+00:00,https://twitter.com/PolskieRadio24/status/8149...,NaN,1466,9836,NaN,1,5,NaN,50301,"['Stoch', 'Oberstdorf', 'Sport', 'PR24', 'TCS'..."
1,08-01-2016,23:02:04,NaN,el_dudkers,Super ten konkurs w Oberstdorfie. Znakomicie P...,2014-05-15 12:31:51+00:00,https://twitter.com/el_dudkers/status/81493475...,NaN,436,3172,NaN,0,0,NaN,121,"['Skoki', 'TCS']"
2,15-01-2016,22:18:03,NaN,atkowska,- udanie dzi w a tak by o prawie ju ...,2014-05-27 17:37:57+00:00,https://twitter.com/atkowska/status/8149248702...,NaN,241,1268,NaN,0,0,NaN,154,"['Oberstdorf', 'tcs', 'skijumping']"
3,15-01-2016,22:18:03,NaN,BikeTeesside,Old standard but always the best quality spare...,2014-04-10 14:47:40+00:00,https://twitter.com/BikeTeesside/status/814915...,Teesside,181,911,[Photo(previewUrl='https://pbs.twimg.com/media...,0,0,NaN,233,"['TCS', 'bikeservice', 'teesside', 'mtb']"
4,19-01-2016,06:03:35,NaN,TCStrength,It's incredible how satisfying throwing heavy ...,2011-02-22 23:23:10+00:00,https://twitter.com/TCStrength/status/81491031...,"Austin, TX",273,5,NaN,0,0,NaN,1690,['tcs']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73798,30-12-2019,22:28:31,NaN,xintae08,year-ender?,2017-03-21 03:34:45+00:00,https://twitter.com/xintae08/status/1211776105...,"Auckland, New Zealand",1289,31729,[Photo(previewUrl='https://pbs.twimg.com/media...,1,0,NaN,1165,"['Tata', 'Taetae']"
73799,30-12-2019,22:28:31,NaN,xintae08,year-ender?,2017-03-21 03:34:45+00:00,https://twitter.com/xintae08/status/1211776105...,"Auckland, New Zealand",1289,31729,[Photo(previewUrl='https://pbs.twimg.com/media...,1,0,NaN,1164,"['Tata', 'Taetae']"
73800,30-12-2019,22:46:29,NaN,chicapotter,"Hoy cumple a os beb Tae, l me rob el co...",2010-03-30 20:28:36+00:00,https://twitter.com/chicapotter/status/1211780...,Puebla,362,2378,NaN,0,0,NaN,80,"['chicapotter', 'sonyoueli', 'army', 'btsarmy'..."
73801,30-12-2019,23:23:41,NaN,balgeun_i,...,2011-05-26 00:36:28+00:00,https://twitter.com/balgeun_i/status/121178998...,ëíë¯¼êµ­,548,26089,[Photo(previewUrl='https://pbs.twimg.com/media...,11,23,NaN,973,"['RJ', 'TATA', 'íë³µíì¸ì']"


In [36]:
! pip install nltk

In [37]:
!pip install pattern

In [38]:
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
import string
import nltk
import pattern
from nltk.corpus import stopwords
#from pattern.en import lemma
from nltk.corpus import stopwords
from collections import  Counter

In [39]:
from pattern.en import lemma

In [40]:
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [41]:
# Function to clean the data(stopwords, punctuations,lowercase,contractions)
def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    #text = [x for x in text if x not in STOPWORDS]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

In [42]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...


True

In [43]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [44]:
data['Tweet'] = data['Tweet'].apply(lambda x :clean_text(x))

In [45]:
data

,Date,Time,Unnamed: 2,User,Tweet,Tweet_created,Url,Location,Friends_count,Favourites_Count,Media,Reply_count,Retweet_count,Retweeted_tweet,Followers_count,Hashtags
0,08-01-2016,23:02:04,NaN,PolskieRadio24,turniej czterech skoczni kamil drugi,2013-12-10 11:31:33+00:00,https://twitter.com/PolskieRadio24/status/8149...,NaN,1466,9836,NaN,1,5,NaN,50301,"['Stoch', 'Oberstdorf', 'Sport', 'PR24', 'TCS'..."
1,08-01-2016,23:02:04,NaN,el_dudkers,super ten konkurs oberstdorfie znakomicie pola...,2014-05-15 12:31:51+00:00,https://twitter.com/el_dudkers/status/81493475...,NaN,436,3172,NaN,0,0,NaN,121,"['Skoki', 'TCS']"
2,15-01-2016,22:18:03,NaN,atkowska,udanie dzi tak by prawie ju lat temu doczekamy...,2014-05-27 17:37:57+00:00,https://twitter.com/atkowska/status/8149248702...,NaN,241,1268,NaN,0,0,NaN,154,"['Oberstdorf', 'tcs', 'skijumping']"
3,15-01-2016,22:18:03,NaN,BikeTeesside,old standard but always the best quality spare...,2014-04-10 14:47:40+00:00,https://twitter.com/BikeTeesside/status/814915...,Teesside,181,911,[Photo(previewUrl='https://pbs.twimg.com/media...,0,0,NaN,233,"['TCS', 'bikeservice', 'teesside', 'mtb']"
4,19-01-2016,06:03:35,NaN,TCStrength,it's incredible how satisfy throw heavy metal ...,2011-02-22 23:23:10+00:00,https://twitter.com/TCStrength/status/81491031...,"Austin, TX",273,5,NaN,0,0,NaN,1690,['tcs']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73798,30-12-2019,22:28:31,NaN,xintae08,year-ender,2017-03-21 03:34:45+00:00,https://twitter.com/xintae08/status/1211776105...,"Auckland, New Zealand",1289,31729,[Photo(previewUrl='https://pbs.twimg.com/media...,1,0,NaN,1165,"['Tata', 'Taetae']"
73799,30-12-2019,22:28:31,NaN,xintae08,year-ender,2017-03-21 03:34:45+00:00,https://twitter.com/xintae08/status/1211776105...,"Auckland, New Zealand",1289,31729,[Photo(previewUrl='https://pbs.twimg.com/media...,1,0,NaN,1164,"['Tata', 'Taetae']"
73800,30-12-2019,22:46:29,NaN,chicapotter,hoy cumple os beb tae me rob el coraz,2010-03-30 20:28:36+00:00,https://twitter.com/chicapotter/status/1211780...,Puebla,362,2378,NaN,0,0,NaN,80,"['chicapotter', 'sonyoueli', 'army', 'btsarmy'..."
73801,30-12-2019,23:23:41,NaN,balgeun_i,,2011-05-26 00:36:28+00:00,https://twitter.com/balgeun_i/status/121178998...,ëíë¯¼êµ­,548,26089,[Photo(previewUrl='https://pbs.twimg.com/media...,11,23,NaN,973,"['RJ', 'TATA', 'íë³µíì¸ì']"


In [46]:
data.sample(10)

,Date,Time,Unnamed: 2,User,Tweet,Tweet_created,Url,Location,Friends_count,Favourites_Count,Media,Reply_count,Retweet_count,Retweeted_tweet,Followers_count,Hashtags
52768,19-11-2018,14:48:07,NaN,SinkoIya,,2017-06-19 06:44:10+00:00,https://twitter.com/SinkoIya/status/1064530768...,NaN,42,4433,[Photo(previewUrl='https://pbs.twimg.com/media...,0,0,NaN,17,"['BTS', 'V', 'Tae', 'Tata', 'taehyung', 'art']"
47814,30-12-2017,17:37:33,NaN,Skijumpingpl,kamil prowadzi przed stefanem hul jak to si sk...,2010-07-12 19:08:52+00:00,https://twitter.com/Skijumpingpl/status/947159...,NaN,488,10833,NaN,0,2,NaN,38793,"['Stoch', 'Oberstdorf', 'skijumpingfamily', 's..."
41018,17-11-2017,15:50:14,NaN,cahdokook,making of ep by via,2016-11-05 16:29:44+00:00,https://twitter.com/cahdokook/status/931550051...,NaN,1154,86099,[Photo(previewUrl='https://pbs.twimg.com/media...,0,0,NaN,1352,"['BT21', 'ì°ì£¼ì¤í', 'ì ëë²ì¤í', 'U..."
28153,20-12-2016,05:45:18,NaN,KaranDhurka,group's the poster boy of indian corp governan...,2013-04-03 21:15:25+00:00,https://twitter.com/KaranDhurka/status/8085503...,Kolkata,49,14,NaN,0,0,NaN,34,['Tata']
12817,08-11-2016,03:55:03,NaN,TCStrength,grab amp go,2011-02-22 23:23:10+00:00,https://twitter.com/TCStrength/status/79416471...,"Austin, TX",273,5,NaN,0,0,NaN,1690,"['tcs', 'traviscountystrength', 'strength', 's..."
39398,08-11-2017,04:18:52,NaN,AcumenWealthca1,out for which be slowly narrow the discount wi...,2017-06-27 09:42:03+00:00,https://twitter.com/AcumenWealthca1/status/928...,NaN,0,0,NaN,0,1,NaN,15,"['Watch', 'Tata', 'motors', 'DVR', 'tata', 'mo..."
38373,30-10-2017,04:41:45,NaN,AmithHegde6,face twintrouble of struggle slow legacy business,2016-01-01 17:34:08+00:00,https://twitter.com/AmithHegde6/status/9248588...,"Bengaluru, India",13550,31946,NaN,0,2,NaN,13682,"['TCS', 'Infosys', 'Wipro', 'digital', 'IoT', ..."
56234,10-12-2018,10:53:30,NaN,Mayu_Chen,purple you,2011-03-12 04:55:47+00:00,https://twitter.com/Mayu_Chen/status/107208186...,å°æ¹¾â æçæäºº,83,1559,NaN,0,0,NaN,17,"['starbucks', 'christmasdrinks', 'milktea', 'B..."
27149,19-12-2016,16:00:54,NaN,business_supser,have lifetime ahead of me but my fight continu...,2016-02-01 19:32:51+00:00,https://twitter.com/business_supser/status/810...,"San Mateo, CA",2,0,NaN,0,0,NaN,2109,['Tata']
38617,03-11-2017,10:11:53,NaN,wondeRM94,bt twitter update wonderm,2017-10-22 03:38:57+00:00,https://twitter.com/wondeRM94/status/926391472...,namjoon's hearteu,160,683,NaN,0,0,NaN,13,"['ì°ì£¼ì¤í', 'ì ëë²ì¤í', 'UNIVERSTA..."


#### Replace emojis with words

In [47]:
data

,Date,Time,Unnamed: 2,User,Tweet,Tweet_created,Url,Location,Friends_count,Favourites_Count,Media,Reply_count,Retweet_count,Retweeted_tweet,Followers_count,Hashtags
0,08-01-2016,23:02:04,NaN,PolskieRadio24,turniej czterech skoczni kamil drugi,2013-12-10 11:31:33+00:00,https://twitter.com/PolskieRadio24/status/8149...,NaN,1466,9836,NaN,1,5,NaN,50301,"['Stoch', 'Oberstdorf', 'Sport', 'PR24', 'TCS'..."
1,08-01-2016,23:02:04,NaN,el_dudkers,super ten konkurs oberstdorfie znakomicie pola...,2014-05-15 12:31:51+00:00,https://twitter.com/el_dudkers/status/81493475...,NaN,436,3172,NaN,0,0,NaN,121,"['Skoki', 'TCS']"
2,15-01-2016,22:18:03,NaN,atkowska,udanie dzi tak by prawie ju lat temu doczekamy...,2014-05-27 17:37:57+00:00,https://twitter.com/atkowska/status/8149248702...,NaN,241,1268,NaN,0,0,NaN,154,"['Oberstdorf', 'tcs', 'skijumping']"
3,15-01-2016,22:18:03,NaN,BikeTeesside,old standard but always the best quality spare...,2014-04-10 14:47:40+00:00,https://twitter.com/BikeTeesside/status/814915...,Teesside,181,911,[Photo(previewUrl='https://pbs.twimg.com/media...,0,0,NaN,233,"['TCS', 'bikeservice', 'teesside', 'mtb']"
4,19-01-2016,06:03:35,NaN,TCStrength,it's incredible how satisfy throw heavy metal ...,2011-02-22 23:23:10+00:00,https://twitter.com/TCStrength/status/81491031...,"Austin, TX",273,5,NaN,0,0,NaN,1690,['tcs']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73798,30-12-2019,22:28:31,NaN,xintae08,year-ender,2017-03-21 03:34:45+00:00,https://twitter.com/xintae08/status/1211776105...,"Auckland, New Zealand",1289,31729,[Photo(previewUrl='https://pbs.twimg.com/media...,1,0,NaN,1165,"['Tata', 'Taetae']"
73799,30-12-2019,22:28:31,NaN,xintae08,year-ender,2017-03-21 03:34:45+00:00,https://twitter.com/xintae08/status/1211776105...,"Auckland, New Zealand",1289,31729,[Photo(previewUrl='https://pbs.twimg.com/media...,1,0,NaN,1164,"['Tata', 'Taetae']"
73800,30-12-2019,22:46:29,NaN,chicapotter,hoy cumple os beb tae me rob el coraz,2010-03-30 20:28:36+00:00,https://twitter.com/chicapotter/status/1211780...,Puebla,362,2378,NaN,0,0,NaN,80,"['chicapotter', 'sonyoueli', 'army', 'btsarmy'..."
73801,30-12-2019,23:23:41,NaN,balgeun_i,,2011-05-26 00:36:28+00:00,https://twitter.com/balgeun_i/status/121178998...,ëíë¯¼êµ­,548,26089,[Photo(previewUrl='https://pbs.twimg.com/media...,11,23,NaN,973,"['RJ', 'TATA', 'íë³µíì¸ì']"


In [48]:
data.sample(10)

,Date,Time,Unnamed: 2,User,Tweet,Tweet_created,Url,Location,Friends_count,Favourites_Count,Media,Reply_count,Retweet_count,Retweeted_tweet,Followers_count,Hashtags
33173,27-12-2016,21:08:38,NaN,UnicableOficial,como tu mi comenzamos el se or de los cielos,2010-02-25 17:30:47+00:00,https://twitter.com/UnicableOficial/status/808...,MÃ©xico,448,31901,[Photo(previewUrl='https://pbs.twimg.com/media...,42,1,NaN,334879,['Tata']
2639,24-07-2016,19:45:52,NaN,tuitekoreruka,,2011-07-10 12:15:38+00:00,https://twitter.com/tuitekoreruka/status/79481...,Tetsu Champion Ship,573,120,[Photo(previewUrl='https://pbs.twimg.com/media...,0,1,NaN,5447,"['TCS', 'éæ¯ç', 'KPF']"
11776,04-11-2016,07:58:56,NaN,business_supser,cyrus mistry's conduct cause enormous harm to ...,2016-02-01 19:32:51+00:00,https://twitter.com/business_supser/status/800...,"San Mateo, CA",2,0,NaN,0,0,NaN,2108,['TCS']
59944,20-09-2019,17:53:53,NaN,Anish_Sharmaaa,here be another gift to the troll brigade befo...,2014-07-03 23:49:22+00:00,https://twitter.com/Anish_Sharmaaa/status/1175...,Delhi,224,6743,[Photo(previewUrl='https://pbs.twimg.com/media...,0,1,NaN,19600,['TCS']
9464,17-10-2016,09:37:40,NaN,pushpender2087,found in tcs build india st s/w r&amp;d in pun...,2012-08-25 06:20:47+00:00,https://twitter.com/pushpender2087/status/7346...,"à¤¦à¤¿à¤²à¥à¤²à¥, à¤­à¤¾à¤°à¤¤",770,592,[Photo(previewUrl='https://pbs.twimg.com/media...,1,1,NaN,434,"['TCS', 'computer']"
58917,29-12-2018,14:54:10,NaN,Eurosport_PL,przed startem sprawd cie jeszcze wpis stefana ...,2010-03-31 11:52:25+00:00,https://twitter.com/Eurosport_PL/status/107902...,Warsaw,356,4877,NaN,0,0,NaN,98193,['TCS']
34198,29-12-2016,11:44:16,NaN,CarKhabri,tata motor hexa spot in the yard of dealer in ...,2011-07-22 12:04:53+00:00,https://twitter.com/CarKhabri/status/806054523...,NaN,394,347,[Photo(previewUrl='https://pbs.twimg.com/media...,0,0,NaN,741,"['TataHexa', 'Tata', 'Hexa']"
23628,13-12-2016,18:53:12,NaN,elmundodemel,por fin junta,2011-08-09 14:40:12+00:00,https://twitter.com/elmundodemel/status/807312...,NaN,475,310,NaN,0,1,NaN,113,"['tata', 'quebienlopasamos']"
23104,13-12-2016,19:40:21,NaN,agk4444,over in,2010-04-07 13:30:26+00:00,https://twitter.com/agk4444/status/80861699263...,"Liverpool, NY",2877,1622,[Photo(previewUrl='https://pbs.twimg.com/media...,0,0,NaN,627,"['Tata', 'Trusts', 'NGOs', 'concerned', 'devel..."
29818,22-12-2016,11:31:19,NaN,arabstodayEN,steel ink deal for talk on uk asset,2011-06-21 10:19:19+00:00,https://twitter.com/arabstodayEN/status/803622...,"London, United Kingdom",4099,22931,NaN,0,0,NaN,2385,"['Tata', 'sale']"


In [49]:
data.sample(10)

,Date,Time,Unnamed: 2,User,Tweet,Tweet_created,Url,Location,Friends_count,Favourites_Count,Media,Reply_count,Retweet_count,Retweeted_tweet,Followers_count,Hashtags
16882,30-11-2016,14:14:47,NaN,CnMetals,china stabilize smm report,2012-12-24 09:03:24+00:00,https://twitter.com/CnMetals/status/7995572521...,Shanghai,253,93,NaN,0,1,NaN,4575,"['Zinc', 'Concentrate', 'TCs']"
26621,18-12-2016,10:10:18,NaN,rocksrakesh07,the statement of cyrus mistry about the tata g...,2013-04-29 23:49:32+00:00,https://twitter.com/rocksrakesh07/status/81184...,Kolkata,1104,3530,NaN,0,0,NaN,3021,['Tata']
26725,18-12-2016,00:05:23,NaN,business_supser,group be in talk with vodafone for consolidati...,2016-02-01 19:32:51+00:00,https://twitter.com/business_supser/status/811...,"San Mateo, CA",2,0,NaN,0,0,NaN,2109,['Tata']
28730,21-12-2016,07:29:37,NaN,business_supser,governance framework need urgent repair cyrus ...,2016-02-01 19:32:51+00:00,https://twitter.com/business_supser/status/806...,"San Mateo, CA",2,0,NaN,0,0,NaN,2109,['Tata']
56508,12-12-2018,11:34:40,NaN,deathrider_7,after more than tweet many mail tata team be n...,2018-12-06 07:38:18+00:00,https://twitter.com/deathrider_7/status/107281...,à¤®à¤¾à¤°à¤µà¤¾à¤¡à¤¼ à¤®à¥à¤ à¤¹à¥ à¤ à¤¿à...,181,533,NaN,1,0,NaN,37,"['shame', 'tata', 'unprofessional']"
56923,15-12-2018,03:33:07,NaN,az16idris,wa.line call,2012-02-14 06:07:41+00:00,https://twitter.com/az16idris/status/107378298...,Surabaya,135,62,NaN,0,0,NaN,21,"['tata', 'tatamotors', 'surabaya', 'waru', 'si..."
68965,09-12-2019,15:00:27,NaN,tuitekoreruka,tetsu champion ship,2011-07-10 12:15:38+00:00,https://twitter.com/tuitekoreruka/status/12040...,Tetsu Champion Ship,573,103,NaN,0,0,NaN,5446,"['TCS', 'TCS13', 'éæ¯ç']"
48106,21-04-2018,09:30:06,NaN,Ventura_Sec,share of consultancy service ltd rise as much ...,2012-12-19 12:28:37+00:00,https://twitter.com/Ventura_Sec/status/9876245...,India,155,1038,[Photo(previewUrl='https://pbs.twimg.com/media...,0,0,NaN,23838,['Tata']
43952,04-12-2017,05:23:53,NaN,Jinaura_1,lo quiero,2012-10-12 19:13:58+00:00,https://twitter.com/Jinaura_1/status/937553019...,VOTACIONES,2185,140091,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0,0,NaN,15330,"['TATA', 'BT21']"
20337,08-12-2016,15:49:31,NaN,mgmt_cons_ser,son serve legal notice to cyrus mistry read more,2016-02-01 19:35:53+00:00,https://twitter.com/mgmt_cons_ser/status/81380...,"San Mateo, CA",2,0,NaN,0,0,NaN,305,['Tata']


In [50]:

data['Tweet'].isnull().sum()

0

In [51]:
# Function to translate various languages to English

def translate(text):

    a = langid.classify(text)

    if a[0] != 'en':

        return GoogleTranslator(source='auto', target='en').translate(text)

    else:

        return text

In [52]:
data['Tweets'] = data['Tweet'].apply(lambda x : translate(x))

ChunkedEncodingError: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))

In [ ]:



contractions = {
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he shall",
"he'll've": "he shall have",
"he's": "he has",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has",
"i'd": "i had",
"i'd've": "i would have",
"i'll": "i shall",
"i'll've": "i shall have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it shall",
"it'll've": "it shall have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she shall",
"she'll've": "she shall have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they shall",
"they'll've": "they shall have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall",
"what'll've": "what shall have",
"what're": "what are",
"what's": "wwhat is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who shall",
"who'll've": "who shall have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you shall",
"you'll've": "you shall have",
"you're": "you are",
"you've": "you have"
}

In [ ]:
def cont_to_exp(x):
    if type(x) is str:
        for key in contractions:
            value=contractions[key]
            x=x.replace(key,value)
        return x
    else:
        return x

In [ ]:
data['Tweets']=data['Tweets'].apply(lambda x: cont_to_exp(x))
data.head()

,Date,Time,Unnamed: 2,User,Tweet,Tweet_created,Url,Location,Friends_count,Favourites_Count,Media,Reply_count,Retweet_count,Retweeted_tweet,Followers_count,Hashtags,Tweets
0,01-01-2016,01:26:47,NaN,up_avi,hny to all,2015-06-27 22:33:14+00:00,https://twitter.com/up_avi/status/682734682769...,"New Delhi, Delhi",340,28420,NaN,0,1,NaN,1220,"['IPO', 'stocks', 'FII', 'DII', 'RIL', 'Sensex...",hny to all
1,01-01-2016,01:26:47,NaN,TruuGawd,happy birthday to the one and only dolla bill ...,2011-10-05 01:31:02+00:00,https://twitter.com/TruuGawd/status/6828095816...,BBU Alum ð,1473,2011,NaN,0,0,NaN,1113,['RIL'],happy birthday to the one and only dolla bill ...
2,01-01-2016,06:24:24,NaN,manojsirsa,what have prompt to interview delhi cm and giv...,2010-03-12 04:09:28+00:00,https://twitter.com/manojsirsa/status/68284735...,à¤­à¤¾à¤°à¤¤,1213,742,NaN,0,6,NaN,31865,"['FalsePropaganda', 'RIL', 'GasDeals', 'NDA', ...",what have prompt to interview delhi cm and giv...
3,01-01-2016,06:24:24,NaN,Petchem_update,kept and price,2015-09-18 05:37:38+00:00,https://twitter.com/Petchem_update/status/6829...,"New Delhi, Delhi",189,0,NaN,0,0,NaN,240,"['RIL', 'PE', 'PP', 'PVC', 'unchanged']",kept and price
4,01-01-2016,08:54:30,NaN,warriorsselfie,love this picture for some reason,2015-12-07 05:04:26+00:00,https://twitter.com/warriorsselfie/status/6829...,NaN,0,5,[Photo(previewUrl='https://pbs.twimg.com/media...,0,0,NaN,146,"['StephenCurry', 'AyeshaCurry', 'DellCurry', '...",love this picture for some reason


In [1]:
from bs4 import BeautifulSoup
import requests, json
resp = requests.get("http://www.netlingo.com/acronyms.php")
soup = BeautifulSoup(resp.text, "html.parser")
slangdict= {}
key=""
value=""
for div in soup.findAll('div', attrs={'class':'list_box3'}):
    for li in div.findAll('li'):
        for a in li.findAll('a'):
            key =a.text
            value = li.text.split(key)[1]
            slangdict[key]=value

with open('myslang.json', 'w') as f:
    json.dump(slangdict, f, indent=2)

In [ ]:
def slang_to_exp(x):
    if type(x) is str:
        for key in slangdict:
            value=slangdict[key]
            x=x.replace(key,value)
        return x
    else:
        return x

In [2]:
print(slangdict)

{'!': 'I have a comment', '#FF': 'Follow Friday', '(U)': 'it means arms around you, hug for you', '*$': 'Starbucks', '**//': 'it means wink wink, nudge nudge', ',!!!!': 'Talk to the hand', '/R/': 'Requesting', '02': 'Your (or my) two cents worth, also seen as m.', '10Q': 'Thank you', '1174': 'Nude club', '121': 'One to one', '123': 'it means I agree', '1337': 'Elite -or- leet -or- L337', '14': 'it refers to the fourteen words', '142n8ly': 'Unfortunately', '143': 'I love you', '1432': 'I Love You Too', '14AA41': 'One for All and All for One', '182': 'I hate you', '187': 'it means murder/ homicide', '19': '0 hand', '1daful': 'it means wonderful', '1V4': 'One Vs. Four', '2': 'it means to, too, two', '20': 'Location', '24/7': 'Twenty Four Seven, as in all the time', '2b': 'To be', '2B or not 2B': 'To Be Or Not To Be', '2b@': 'To Be At', '2BZ4UQT': 'Too Busy For You Cutey', '2B~not2B': 'To be or not to be', '2d4': 'To die for', '2day': 'Today', '2DLoo': 'Toodle oo', '2G2B4G': 'Too Good To B

In [ ]:
data['clean_tweets']=data['Tweets'].apply(lambda x: slang_to_exp(x))
data.head()

,Date,Time,Unnamed: 2,User,Tweet,Tweet_created,Url,Location,Friends_count,Favourites_Count,Media,Reply_count,Retweet_count,Retweeted_tweet,Followers_count,Hashtags,Tweets,clean_tweets
0,01-01-2016,01:26:47,NaN,up_avi,hny to all,2015-06-27 22:33:14+00:00,https://twitter.com/up_avi/status/682734682769...,"New Delhi, Delhi",340,28420,NaN,0,1,NaN,1220,"['IPO', 'stocks', 'FII', 'DII', 'RIL', 'Sensex...",hny to all,hny to all
1,01-01-2016,01:26:47,NaN,TruuGawd,happy birthday to the one and only dolla bill ...,2011-10-05 01:31:02+00:00,https://twitter.com/TruuGawd/status/6828095816...,BBU Alum ð,1473,2011,NaN,0,0,NaN,1113,['RIL'],happy birthday to the one and only dolla bill ...,happy biaththeaey to the oAny and only dolla b...
2,01-01-2016,06:24:24,NaN,manojsirsa,what have prompt to interview delhi cm and giv...,2010-03-12 04:09:28+00:00,https://twitter.com/manojsirsa/status/68284735...,à¤­à¤¾à¤°à¤¤,1213,742,NaN,0,6,NaN,31865,"['FalsePropaganda', 'RIL', 'GasDeals', 'NDA', ...",what have prompt to interview delhi cm and giv...,what have paompt to inteaview delhi cm and giv...
3,01-01-2016,06:24:24,NaN,Petchem_update,kept and price,2015-09-18 05:37:38+00:00,https://twitter.com/Petchem_update/status/6829...,"New Delhi, Delhi",189,0,NaN,0,0,NaN,240,"['RIL', 'PE', 'PP', 'PVC', 'unchanged']",kept and price,kept and paice
4,01-01-2016,08:54:30,NaN,warriorsselfie,love this picture for some reason,2015-12-07 05:04:26+00:00,https://twitter.com/warriorsselfie/status/6829...,NaN,0,5,[Photo(previewUrl='https://pbs.twimg.com/media...,0,0,NaN,146,"['StephenCurry', 'AyeshaCurry', 'DellCurry', '...",love this picture for some reason,it means helve this pictWhy? -or- oae foa some...


In [ ]:
data.head()

,Date,Time,Unnamed: 2,User,Tweet,Tweet_created,Url,Location,Friends_count,Favourites_Count,Media,Reply_count,Retweet_count,Retweeted_tweet,Followers_count,Hashtags,Tweets,clean_tweets
0,01-01-2016,01:26:47,NaN,up_avi,hny to all,2015-06-27 22:33:14+00:00,https://twitter.com/up_avi/status/682734682769...,"New Delhi, Delhi",340,28420,NaN,0,1,NaN,1220,"['IPO', 'stocks', 'FII', 'DII', 'RIL', 'Sensex...",hny to all,hny to all
1,01-01-2016,01:26:47,NaN,TruuGawd,happy birthday to the one and only dolla bill ...,2011-10-05 01:31:02+00:00,https://twitter.com/TruuGawd/status/6828095816...,BBU Alum ð,1473,2011,NaN,0,0,NaN,1113,['RIL'],happy birthday to the one and only dolla bill ...,happy biaththeaey to the oAny and only dolla b...
2,01-01-2016,06:24:24,NaN,manojsirsa,what have prompt to interview delhi cm and giv...,2010-03-12 04:09:28+00:00,https://twitter.com/manojsirsa/status/68284735...,à¤­à¤¾à¤°à¤¤,1213,742,NaN,0,6,NaN,31865,"['FalsePropaganda', 'RIL', 'GasDeals', 'NDA', ...",what have prompt to interview delhi cm and giv...,what have paompt to inteaview delhi cm and giv...
3,01-01-2016,06:24:24,NaN,Petchem_update,kept and price,2015-09-18 05:37:38+00:00,https://twitter.com/Petchem_update/status/6829...,"New Delhi, Delhi",189,0,NaN,0,0,NaN,240,"['RIL', 'PE', 'PP', 'PVC', 'unchanged']",kept and price,kept and paice
4,01-01-2016,08:54:30,NaN,warriorsselfie,love this picture for some reason,2015-12-07 05:04:26+00:00,https://twitter.com/warriorsselfie/status/6829...,NaN,0,5,[Photo(previewUrl='https://pbs.twimg.com/media...,0,0,NaN,146,"['StephenCurry', 'AyeshaCurry', 'DellCurry', '...",love this picture for some reason,it means helve this pictWhy? -or- oae foa some...


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# initiating the VADER module
analyser = SentimentIntensityAnalyzer()
# Function to calculate sentiment score
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    return score
# Function to calculate compound score
def compound_score(text):
    comp=sentiment_analyzer_scores(text)
    return comp['compound'] # returns the compound score from the dictionary

In [ ]:
data.to_csv('RIL2016preprocessed.csv')

In [ ]:
import os
os.getcwd()

'/content'